In [11]:
import os
import sys
# Add the scripts folder to the Python path
sys.path.append(os.path.abspath("../scripts"))

In [12]:
import yfinance as yf 
import pandas as pd 
import numpy as np 
import matplotlib. pyplot as plt 
from datetime import datetime 
plt.style.use('fivethirtyeight')

In [13]:
end=datetime.now()
start=datetime(end.year-15,end.month,end.day)
stock=('BTC-USD')
stock_data=yf.download(stock,start=start,end=end)

[*********************100%***********************]  1 of 1 completed


In [14]:
#closing price history
closing_price=stock_data[['Close']]

In [17]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(closing_price[['Close']].dropna())